In [8]:
import pandas as pd
import requests as rq
import utils.constants as constants
import streamlit as st

url1 = constants.RDW_KENTEKEN_URL
url2 = constants.RDW_BRANDSTOF_API_URL

json_kenteken = rq.get(url1, 
headers={"X-App-Token": st.secrets['RDW_API_KEY']}, params={"$limit": 100_000}).json()

json_brandstof = rq.get(url2, 
headers={"X-App-Token": st.secrets['RDW_API_KEY']}, params={"$limit": 100_000}).json()

df1 = pd.DataFrame.from_dict(json_kenteken)
df2 = pd.DataFrame.from_dict(json_brandstof)

df = pd.merge(df1, df2, on='kenteken')

print(len(df))
# df1['kenteken'] = df1['kenteken'].astype(str)
# df2['kenteken'] = df2['kenteken'].astype(str)

# df1.info()
# print(df1)
# print(df2)


# response = rq.get(constants.OCM_API_URL, 
# headers={"X-API-Key": st.secrets['OCM_API_KEY']}).json()
# pd.DataFrame.from_dict(response).head(5)

MemoryError: 

In [74]:
def fetchOCMData(isFilters=False):
    response = rq.get("https://api.openchargemap.io/v3/poi/?output=json&countrycode=NL&verbose=false&includecomments=false", 
    headers={"X-API-Key": st.secrets['OCM_API_KEY']}).json()
    df = pd.json_normalize(response)
    fitlers = pd.json_normalize(response, record_path=["Connections"])
    listOfCols = ['IsRecentlyVerified', 'UsageType.Title', 'AddressInfo.Latitude', 'AddressInfo.Longitude', 'Connections', 'NumberOfPoints', 'StatusTypeID']
    df['Connections'] = df['Connections'].apply(lambda x: [y['ConnectionType']['Title'] for y in x])
    df.drop(columns=df.columns[~df.columns.isin(listOfCols)].values, axis=1, inplace=True)
    df = df[df['IsRecentlyVerified']]
    if (isFilters):
        return fitlers['ConnectionType.Title'].unique()
    return df

df = fetchOCMData()

df.head()

,IsRecentlyVerified,Connections,NumberOfPoints,StatusTypeID,UsageType.Title,AddressInfo.Latitude,AddressInfo.Longitude
0,True,"[CHAdeMO, CCS (Type 2)]",2,50,Public - Membership Required,52.154843,6.201008
1,True,[Type 2 (Socket Only)],2,50,Public - Membership Required,52.220917,4.531350
2,True,[Type 2 (Socket Only)],1,50,Public - Membership Required,51.411700,6.167800
3,True,[Type 2 (Socket Only)],1,50,Public,52.042242,5.665901
4,True,[Type 2 (Socket Only)],2,50,"Private - For Staff, Visitors or Customers",52.399358,6.783781
